<a href="https://colab.research.google.com/github/bet-lbm/pinto/blob/master/train_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/bet-lbm/dl-dataset.git

Cloning into 'dl-dataset'...
remote: Enumerating objects: 3028, done.
remote: Counting objects: 100% (3028/3028), done.
remote: Compressing objects: 100% (3027/3027), done.
remote: Total 3028 (delta 1), reused 3028 (delta 1), pack-reused 0
Receiving objects: 100% (3028/3028), 63.02 MiB | 10.22 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [0]:
import sys
import os
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import  Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as K

In [0]:
# limpiar seccion de tensorflow
K.clear_session()

#rutas de DataSet
data_entrenamiento = './dl-dataset/train'
data_validacion = './dl-dataset/test'

In [0]:
# parametros
epocas = 20 # numero de veces que vamos a iterar
longitud, altura = 150, 150 #tamaño de procesamiento de nuestras imagenes 
batch_size = 32 #numero de imagenes que vamos a mandar a procesar en cada uno de los pasos
pasos = 1000 #numero de veces que se va a procesar la informacion en cada una de la epocas
validation_steps = 200 #pasos de validacion 
filtroConv1 = 32 #filtros en la convolucion el numero de filtros
filtroConv2 = 64 
tamano_filtro1 = (3, 3) #tamaño del filtro altura de 3 y longitud de 3
tamano_filtro2 = (2, 2)  #tamaño del filtro altura de 3 y longitud de 3
tamano_pool = (2, 2) 
clases = 3  # gato perro gorila
lr = 0.0005 # ajustes de la red neuronal para acercarse para una solucion optima numero pequeño

#pre procesamiento de imagenes 
entrenamieto_datagen = ImageDataGenerator(
    rescale=1./255, # rescala cada uno de los pixeles tiene un rango de 0 a 255 => los valores de pixeles de 0 a 1
    shear_range=0.3, #genera imagenes y las inclina 
    zoom_range=0.3, #realiza zoom a la imagenes
    horizontal_flip=True #toma imagen y la invierte
)

validacion_datagen=ImageDataGenerator(
    rescale=1./255
)

#generar la imagenes que vamos a usar para entrenar con preprocesamiento 
imagen_entrenamiento = entrenamieto_datagen.flow_from_directory(
    data_entrenamiento,# entra en el directorio de entrenamiento
    target_size=(altura, longitud),# va a procesar a una altura y longitud especifica 
    batch_size= batch_size, # va procesar en un batch de 32
    class_mode='categorical' # va a procesar como categoria etiquetas perro , gato, gorila
)
imagen_validacion = validacion_datagen.flow_from_directory(
    data_validacion,
    target_size = (altura, longitud),
    batch_size = batch_size,
    class_mode= 'categorical'
)

#Creamos nuestra red convolucional
cnn = Sequential() # nuestra red nueronal es sequencial varias capas apiladas

cnn.add(Convolution2D(filtroConv1,tamano_filtro1,padding='same',input_shape=(altura,longitud,3),activation='relu'))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Convolution2D(filtroConv2,tamano_filtro2,padding='same'))
cnn.add(MaxPooling2D(pool_size=tamano_pool))
#empezamos nuestra clasificacion 
cnn.add(Flatten())#imagen muy profunda pero muy pequeña se vuelve plana en una sola dimension que tiene la informacion de nuestra red neuronal
cnn.add(Dense(256,activation='relu'))#le decimos que va a tener 256 neuronas y de activacion relu
cnn.add(Dropout(0.5))# a la capa Dense vamos a apagarle el 50% de las neuronas en cada paso esto se hace  para evitar sobre ajustar
cnn.add(Dense(clases, activation='softmax')) # solo 3 neuronas y softmax nos ayuda a decir que esta imagen que la imagen que me 
 #diste es 80% de probabilidad que sea perro,15% que sea gato y 5% que sea gorila 

#durante el entrenamiento nuestra funcion de perdida es categorical_crossentropy, el optimizador es Adam un un leraning read lr=0.0005
# y la metrica es accuracy que es que tan bien esta aprendiendo nuestra red nueronal
cnn.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=lr), metrics=['accuracy']) 
#vamos a entrenar con imagen de entrenamiento, vamos a correrlo cada epoca con 1000 pasos, el numero de epocas, nuestras imagenes de validacion, tantos pasos de validadion va a correr por cada epoca
cnn.fit(imagen_entrenamiento,steps_per_epoch=pasos, epochs=epocas, validation_data=imagen_validacion, validation_steps=validation_steps)


Found 999 images belonging to 3 classes.
Found 2043 images belonging to 3 classes.
Epoch 1/20
1000/1000 [==============================] - 288s 288ms/step - loss: 0.6751 - acc: 0.6747 - val_loss: 0.6316 - val_acc: 0.7056
Epoch 2/20
1000/1000 [==============================] - 283s 283ms/step - loss: 0.3563 - acc: 0.8440 - val_loss: 0.8322 - val_acc: 0.7071
Epoch 3/20
1000/1000 [==============================] - 282s 282ms/step - loss: 0.2158 - acc: 0.9124 - val_loss: 1.0079 - val_acc: 0.6912
Epoch 4/20
1000/1000 [==============================] - 282s 282ms/step - loss: 0.1381 - acc: 0.9484 - val_loss: 1.0355 - val_acc: 0.7157
Epoch 5/20
1000/1000 [==============================] - 282s 282ms/step - loss: 0.1012 - acc: 0.9633 - val_loss: 1.2445 - val_acc: 0.7165
Epoch 6/20
1000/1000 [==============================] - 281s 281ms/step - loss: 0.0891 - acc: 0.9676 - val_loss: 1.3995 - val_acc: 0.7193
Epoch 7/20
1000/1000 [==============================] - 283s 283ms/step - loss: 0.0837 - 

In [0]:
#guarda nuestro modelo en un archivo
dir='./modelo'
if not os.path.exists(dir):
    os.mkdir(dir)
cnn.save('./modelo/modelo.h5')
cnn.save_weights('./modelo/pesos.h5')

# `Predicción`

In [0]:
import os
os.getcwd()

'/content'